In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from ADF_test import is_stationary
import pmdarima as pm
from pmdarima.model_selection import train_test_split
import numpy as np
from sklearn.metrics import r2_score
from statsmodels.tsa.seasonal import seasonal_decompose
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = 'plotly_white'
plot_template = dict(
    layout=go.Layout({
        'font_size': 8,
        'xaxis_title_font_size': 8,
        'yaxis_title_font_size': 8,
        }   
))
hydro_stations = ['Tangnaihai','Guide','Xunhua']

In [4]:
for hydro_station in hydro_stations:
    df = pd.read_csv(f'../data/{hydro_station.lower()}_natural_monthly_flow.csv', parse_dates=['date'], index_col='date')
    # compute the average flow before 2014-12-31
    df_avg_before_2014 = df.loc[:'2014-12-31','flow(m^3/s)'].mean()
    df.info()
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df.index, y=df['flow(m^3/s)'], mode='lines', name=f'{hydro_station} River'))
    fig.add_shape(
        type="line",
        x0=df.index[0],
        y0=df_avg_before_2014,
        x1=df.index[-1],
        y1=df_avg_before_2014,
        line=dict(color="Red", width=2)
    )
    fig.update_layout(title=f'Yellow River Monthly Flow at {hydro_station}',
                       xaxis_title='Date',
                       yaxis_title='Flow (m^3/s)')
    fig.show()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 840 entries, 1950-01-01 to 2019-12-01
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   flow(m^3/s)  840 non-null    float64
dtypes: float64(1)
memory usage: 45.4 KB


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 756 entries, 1957-01-01 to 2019-12-01
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   flow(m^3/s)  756 non-null    float64
dtypes: float64(1)
memory usage: 28.0 KB


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 756 entries, 1957-01-01 to 2019-12-01
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   flow(m^3/s)  756 non-null    float64
dtypes: float64(1)
memory usage: 28.0 KB


In [41]:
val_years = np.arange(2009,2015)
test_obs = df.loc["2009-01-01":"2014-12-31",:]
for hydro_station in hydro_stations:
    df = pd.read_csv(f'../data/{hydro_station.lower()}_natural_monthly_flow.csv', parse_dates=['date'], index_col='date')
    modes = [
        'multiplicative',
        # 'additive'
    ]
    for mode in modes:
        test_pred = pd.DataFrame()
        train_r2_list = []
        for year in val_years:
            train = df.loc[:"{}-12-31".format(year-1),:]
            test = df.loc["{}-01-01".format(year):"{}-12-31".format(year),:]
            decomposition = seasonal_decompose(train['flow(m^3/s)'], model=mode, period=12,extrapolate_trend=1)
            trend = decomposition.trend
            seasonal = decomposition.seasonal
            residual = decomposition.resid
            # fit the ARIMA model to the trend component
            model = pm.auto_arima(trend, 
                                  seasonal=False, 
                                  information_criterion='bic',
                                  trace=True,
                                  error_action='ignore',  
                                  suppress_warnings=True, 
                                  stepwise=True)
            model.summary()
            train_trend_pred = model.predict_in_sample()
            test_trend_pred = model.predict(n_periods=len(test))
            # fit the ARIMA model to the seasonal component
            model = pm.auto_arima(seasonal, 
                                  m = 12,
                                  seasonal_test = 'ch',
                                  seasonal=True, 
                                  information_criterion='bic',
                                  trace=True,
                                  error_action='ignore',  
                                  suppress_warnings=True, 
                                  stepwise=True)
            model.summary()
            train_seasonal_pred = model.predict_in_sample()
            test_seasonal_pred = model.predict(n_periods=len(test))
            # fit the ARIMA model to the residual component
            model = pm.auto_arima(residual, 
                                  seasonal=False, 
                                  information_criterion='bic',
                                  trace=True,
                                  error_action='ignore',  
                                  suppress_warnings=True, 
                                  stepwise=True)
            model.summary()
            train_residual_pred = model.predict_in_sample()
            test_residual_pred = model.predict(n_periods=len(test))
            # combine the trend, seasonal, and residual components
            if mode == 'multiplicative':
                train_pred_year = train_trend_pred * train_seasonal_pred * train_residual_pred
                test_pred_year = test_trend_pred * test_seasonal_pred * test_residual_pred
            else:
                train_pred_year = train_trend_pred + train_seasonal_pred + train_residual_pred
                test_pred_year = test_trend_pred + test_seasonal_pred + test_residual_pred

            train_pred_year.index = train.index
            train_pred_year.name = "SimFlow(m^3/s)"
            train_result = pd.concat([train,train_pred_year],axis=1)
            train_result.to_csv(f'../result/ARIMAPredData/seasonal_decompose_{mode}_arima_train_sim_{hydro_station.lower()}_before_{year}.csv')
            train_r2 = r2_score(train['flow(m^3/s)'], train_pred_year)
            train_r2_list.append(train_r2)
            test_pred_year.index = test.index
            test_pred_year.name = "flow(m^3/s)"
            test_pred = pd.concat([test_pred,test_pred_year])

        print(sum(train_r2_list)/len(train_r2_list))
        print(test_pred)
        print(r2_score(test_obs,test_pred)) # 0.6934 multiplicative;0.5988 additive
        test_pred.index.name = 'date'
        test_pred.to_csv(f'../result/ARIMAPredData/seasonal_decompose_{mode}_arima_pred_{hydro_station.lower()}_{val_years[0]}_{val_years[-1]}.csv')

        # visualize the results using plotly
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=test_obs.index, y=test_obs['flow(m^3/s)'], mode='lines', name='Observed'))
        fig.add_trace(go.Scatter(x=test_pred.index, y=test_pred['flow(m^3/s)'], mode='lines', name='Predicted'))
        fig.show()


Performing stepwise search to minimize bic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : BIC=inf, Time=0.38 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : BIC=6773.615, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : BIC=6217.637, Time=0.05 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : BIC=inf, Time=0.10 sec
 ARIMA(0,1,0)(0,0,0)[0]             : BIC=6767.055, Time=0.02 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : BIC=6084.398, Time=0.08 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : BIC=6008.107, Time=0.10 sec
 ARIMA(4,1,0)(0,0,0)[0] intercept   : BIC=5974.998, Time=0.14 sec
 ARIMA(5,1,0)(0,0,0)[0] intercept   : BIC=5949.048, Time=0.15 sec
 ARIMA(5,1,1)(0,0,0)[0] intercept   : BIC=inf, Time=0.57 sec
 ARIMA(4,1,1)(0,0,0)[0] intercept   : BIC=inf, Time=0.49 sec
 ARIMA(5,1,0)(0,0,0)[0]             : BIC=5942.487, Time=0.11 sec
 ARIMA(4,1,0)(0,0,0)[0]             : BIC=5968.437, Time=0.09 sec
 ARIMA(5,1,1)(0,0,0)[0]             : BIC=inf, Time=0.44 sec
 ARIMA(4,1,1)(0,0,0)[0]             : BIC=inf, Time=0.32 s

C:\Users\ZJY\AppData\Local\Temp\ipykernel_39020\2992677552.py:69: FutureWarning:

The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.



 ARIMA(2,1,2)(0,0,0)[0] intercept   : BIC=inf, Time=0.43 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : BIC=6887.611, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : BIC=6314.784, Time=0.05 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : BIC=inf, Time=0.10 sec
 ARIMA(0,1,0)(0,0,0)[0]             : BIC=6881.084, Time=0.02 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : BIC=6177.100, Time=0.08 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : BIC=6097.139, Time=0.10 sec
 ARIMA(4,1,0)(0,0,0)[0] intercept   : BIC=6061.071, Time=0.14 sec
 ARIMA(5,1,0)(0,0,0)[0] intercept   : BIC=6030.610, Time=0.18 sec
 ARIMA(5,1,1)(0,0,0)[0] intercept   : BIC=inf, Time=0.59 sec
 ARIMA(4,1,1)(0,0,0)[0] intercept   : BIC=inf, Time=0.50 sec
 ARIMA(5,1,0)(0,0,0)[0]             : BIC=6024.043, Time=0.10 sec
 ARIMA(4,1,0)(0,0,0)[0]             : BIC=6054.508, Time=0.08 sec
 ARIMA(5,1,1)(0,0,0)[0]             : BIC=inf, Time=0.48 sec
 ARIMA(4,1,1)(0,0,0)[0]             : BIC=inf, Time=0.39 sec

Best model:  ARIMA(5,1,0)(0,0,0)[0]    

Performing stepwise search to minimize bic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : BIC=inf, Time=0.38 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : BIC=6773.615, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : BIC=6217.637, Time=0.05 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : BIC=inf, Time=0.10 sec
 ARIMA(0,1,0)(0,0,0)[0]             : BIC=6767.055, Time=0.03 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : BIC=6084.398, Time=0.08 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : BIC=6008.107, Time=0.10 sec
 ARIMA(4,1,0)(0,0,0)[0] intercept   : BIC=5974.998, Time=0.14 sec
 ARIMA(5,1,0)(0,0,0)[0] intercept   : BIC=5949.048, Time=0.15 sec
 ARIMA(5,1,1)(0,0,0)[0] intercept   : BIC=inf, Time=0.56 sec
 ARIMA(4,1,1)(0,0,0)[0] intercept   : BIC=inf, Time=0.48 sec
 ARIMA(5,1,0)(0,0,0)[0]             : BIC=5942.487, Time=0.09 sec
 ARIMA(4,1,0)(0,0,0)[0]             : BIC=5968.437, Time=0.08 sec
 ARIMA(5,1,1)(0,0,0)[0]             : BIC=inf, Time=0.43 sec
 ARIMA(4,1,1)(0,0,0)[0]             : BIC=inf, Time=0.34 s

C:\Users\ZJY\AppData\Local\Temp\ipykernel_39020\2992677552.py:69: FutureWarning:

The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.



 ARIMA(2,1,2)(0,0,0)[0] intercept   : BIC=inf, Time=0.52 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : BIC=6887.611, Time=0.03 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : BIC=6314.784, Time=0.06 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : BIC=inf, Time=0.12 sec
 ARIMA(0,1,0)(0,0,0)[0]             : BIC=6881.084, Time=0.02 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : BIC=6177.100, Time=0.08 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : BIC=6097.139, Time=0.10 sec
 ARIMA(4,1,0)(0,0,0)[0] intercept   : BIC=6061.071, Time=0.14 sec
 ARIMA(5,1,0)(0,0,0)[0] intercept   : BIC=6030.610, Time=0.18 sec
 ARIMA(5,1,1)(0,0,0)[0] intercept   : BIC=inf, Time=0.62 sec
 ARIMA(4,1,1)(0,0,0)[0] intercept   : BIC=inf, Time=0.47 sec
 ARIMA(5,1,0)(0,0,0)[0]             : BIC=6024.043, Time=0.09 sec
 ARIMA(4,1,0)(0,0,0)[0]             : BIC=6054.508, Time=0.08 sec
 ARIMA(5,1,1)(0,0,0)[0]             : BIC=inf, Time=0.51 sec
 ARIMA(4,1,1)(0,0,0)[0]             : BIC=inf, Time=0.54 sec

Best model:  ARIMA(5,1,0)(0,0,0)[0]    

In [36]:
pred_years = [2015,2016,2017,2018,2019]
test_obs = df.loc["2015-01-01":"2019-12-31",:]
for hydro_station in hydro_stations:
    df = pd.read_csv(f'../data/{hydro_station.lower()}_natural_monthly_flow.csv', parse_dates=['date'], index_col='date')
    modes = [
        'multiplicative',
        # 'additive'
    ]
    for mode in modes:
        test_pred = pd.DataFrame()
        train_r2_list = []
        for year in pred_years:
            train = df.loc[:"{}-12-31".format(year-1),:]
            test = df.loc["{}-01-01".format(year):"{}-12-31".format(year),:]
            # decompose the data using seasonal_decompose
            decomposition = seasonal_decompose(train['flow(m^3/s)'], model=mode, period=12,extrapolate_trend=1)
            # decomposition = seasonal_decompose(train['flow(m^3/s)'], model='additive', period=12,extrapolate_trend=1)
            # extract the trend component
            trend = decomposition.trend
            seasonal = decomposition.seasonal
            residual = decomposition.resid
            # fit the ARIMA model to the trend component
            model = pm.auto_arima(trend, 
                                  seasonal=False, 
                                  information_criterion='bic',
                                  trace=True,
                                  error_action='ignore',  
                                  suppress_warnings=True, 
                                  stepwise=True)
            model.summary()
            train_trend_pred = model.predict_in_sample()
            test_trend_pred = model.predict(n_periods=len(test))
            # fit the ARIMA model to the seasonal component
            model = pm.auto_arima(seasonal, 
                                  m = 12,
                                  seasonal_test = 'ch',
                                  seasonal=True, 
                                  information_criterion='bic',
                                  trace=True,
                                  error_action='ignore',  
                                  suppress_warnings=True, 
                                  stepwise=True)
            model.summary()
            train_seasonal_pred = model.predict_in_sample()
            test_seasonal_pred = model.predict(n_periods=len(test))
            # fit the ARIMA model to the residual component
            model = pm.auto_arima(residual, 
                                  seasonal=False, 
                                  information_criterion='bic',
                                  trace=True,
                                  error_action='ignore',  
                                  suppress_warnings=True, 
                                  stepwise=True)
            model.summary()
            train_residual_pred = model.predict_in_sample()
            test_residual_pred = model.predict(n_periods=len(test))
            # combine the trend, seasonal, and residual components
            if mode == 'multiplicative':
                train_pred_year = train_trend_pred * train_seasonal_pred * train_residual_pred
                test_pred_year = test_trend_pred * test_seasonal_pred * test_residual_pred
            else:
                train_pred_year = train_trend_pred + train_seasonal_pred + train_residual_pred
                test_pred_year = test_trend_pred + test_seasonal_pred + test_residual_pred

            train_pred_year.index = train.index
            train_pred_year.name = "SimFlow(m^3/s)"
            train_result = pd.concat([train,train_pred_year],axis=1)
            train_result.to_csv(f'../result/ARIMAPredData/seasonal_decompose_{mode}_arima_train_sim_{hydro_station.lower()}_before_{year}.csv')
            train_r2 = r2_score(train['flow(m^3/s)'], train_pred_year)
            train_r2_list.append(train_r2)
            test_pred_year.index = test.index
            test_pred_year.name = "flow(m^3/s)"
            test_pred = pd.concat([test_pred,test_pred_year])

        print(sum(train_r2_list)/len(train_r2_list))
        print(r2_score(test_obs,test_pred)) # 0.6934 multiplicative;0.5988 additive
        test_pred.index.name = 'date'
        test_pred.to_csv(f'../result/ARIMAPredData/seasonal_decompose_{mode}_arima_pred_{hydro_station.lower()}_{pred_years[0]}_{pred_years[-1]}.csv')

        # visualize the results using plotly
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=test_obs.index, y=test_obs['flow(m^3/s)'], mode='lines', name='Observed'))
        fig.add_trace(go.Scatter(x=test_pred.index, y=test_pred['flow(m^3/s)'], mode='lines', name='Predicted'))
        fig.show()

        # Best model:  ARIMA(5,0,1)(0,0,0)[0] intercept m=1
        # Total fit time: 5.661 seconds
        # 0.8163614700601103
        # 0.6934717968558676

        # Best model:  ARIMA(5,0,0)(0,0,0)[0] intercept m=6
        # Total fit time: 4.456 seconds
        # 0.8403380825495306
        # 0.7699084203586042
    


date
1950-01-01    705.294220
1950-02-01    700.628644
1950-03-01    695.963068
1950-04-01    691.297491
1950-05-01    686.631915
                 ...    
2014-08-01    672.024750
2014-09-01    692.477439
2014-10-01    712.930127
2014-11-01    733.382815
2014-12-01    753.835503
Name: trend, Length: 780, dtype: float64
Performing stepwise search to minimize bic
 ARIMA(2,0,2)(0,0,0)[0]             : BIC=inf, Time=0.22 sec
 ARIMA(0,0,0)(0,0,0)[0]             : BIC=12336.621, Time=0.02 sec
 ARIMA(1,0,0)(0,0,0)[0]             : BIC=inf, Time=0.02 sec
 ARIMA(0,0,1)(0,0,0)[0]             : BIC=inf, Time=0.07 sec
 ARIMA(1,0,1)(0,0,0)[0]             : BIC=inf, Time=0.11 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : BIC=10127.252, Time=0.02 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : BIC=7476.811, Time=0.06 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept   : BIC=6823.427, Time=0.11 sec
 ARIMA(3,0,0)(0,0,0)[0] intercept   : BIC=6702.917, Time=0.17 sec
 ARIMA(4,0,0)(0,0,0)[0] intercept   : BIC=6590.523, Time=0.1

C:\Users\ZJY\AppData\Local\Temp\ipykernel_39020\620897473.py:71: FutureWarning:

The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.



 ARIMA(2,0,2)(0,0,0)[0]             : BIC=inf, Time=0.22 sec
 ARIMA(0,0,0)(0,0,0)[0]             : BIC=12519.403, Time=0.02 sec
 ARIMA(1,0,0)(0,0,0)[0]             : BIC=inf, Time=0.03 sec
 ARIMA(0,0,1)(0,0,0)[0]             : BIC=inf, Time=0.06 sec
 ARIMA(1,0,1)(0,0,0)[0]             : BIC=inf, Time=0.12 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : BIC=10284.412, Time=0.02 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : BIC=7586.987, Time=0.06 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept   : BIC=6925.835, Time=0.11 sec
 ARIMA(3,0,0)(0,0,0)[0] intercept   : BIC=6802.647, Time=0.20 sec
 ARIMA(4,0,0)(0,0,0)[0] intercept   : BIC=6687.415, Time=0.17 sec
 ARIMA(5,0,0)(0,0,0)[0] intercept   : BIC=6661.356, Time=0.32 sec
 ARIMA(5,0,1)(0,0,0)[0] intercept   : BIC=inf, Time=0.58 sec
 ARIMA(4,0,1)(0,0,0)[0] intercept   : BIC=inf, Time=0.44 sec
 ARIMA(5,0,0)(0,0,0)[0]             : BIC=inf, Time=0.10 sec

Best model:  ARIMA(5,0,0)(0,0,0)[0] intercept
Total fit time: 2.455 seconds
Performing stepwise search to mi

date
1950-01-01    705.294220
1950-02-01    700.628644
1950-03-01    695.963068
1950-04-01    691.297491
1950-05-01    686.631915
                 ...    
2014-08-01    672.024750
2014-09-01    692.477439
2014-10-01    712.930127
2014-11-01    733.382815
2014-12-01    753.835503
Name: trend, Length: 780, dtype: float64
Performing stepwise search to minimize bic
 ARIMA(2,0,2)(0,0,0)[0]             : BIC=inf, Time=0.24 sec
 ARIMA(0,0,0)(0,0,0)[0]             : BIC=12336.621, Time=0.02 sec
 ARIMA(1,0,0)(0,0,0)[0]             : BIC=inf, Time=0.02 sec
 ARIMA(0,0,1)(0,0,0)[0]             : BIC=inf, Time=0.07 sec
 ARIMA(1,0,1)(0,0,0)[0]             : BIC=inf, Time=0.12 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : BIC=10127.252, Time=0.03 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : BIC=7476.811, Time=0.06 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept   : BIC=6823.427, Time=0.17 sec
 ARIMA(3,0,0)(0,0,0)[0] intercept   : BIC=6702.917, Time=0.21 sec
 ARIMA(4,0,0)(0,0,0)[0] intercept   : BIC=6590.523, Time=0.1

C:\Users\ZJY\AppData\Local\Temp\ipykernel_39020\620897473.py:71: FutureWarning:

The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.



 ARIMA(2,0,2)(0,0,0)[0]             : BIC=inf, Time=0.24 sec
 ARIMA(0,0,0)(0,0,0)[0]             : BIC=12519.403, Time=0.03 sec
 ARIMA(1,0,0)(0,0,0)[0]             : BIC=inf, Time=0.03 sec
 ARIMA(0,0,1)(0,0,0)[0]             : BIC=inf, Time=0.07 sec
 ARIMA(1,0,1)(0,0,0)[0]             : BIC=inf, Time=0.13 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : BIC=10284.412, Time=0.03 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : BIC=7586.987, Time=0.07 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept   : BIC=6925.835, Time=0.14 sec
 ARIMA(3,0,0)(0,0,0)[0] intercept   : BIC=6802.647, Time=0.23 sec
 ARIMA(4,0,0)(0,0,0)[0] intercept   : BIC=6687.415, Time=0.21 sec
 ARIMA(5,0,0)(0,0,0)[0] intercept   : BIC=6661.356, Time=0.34 sec
 ARIMA(5,0,1)(0,0,0)[0] intercept   : BIC=inf, Time=0.61 sec
 ARIMA(4,0,1)(0,0,0)[0] intercept   : BIC=inf, Time=0.36 sec
 ARIMA(5,0,0)(0,0,0)[0]             : BIC=inf, Time=0.10 sec

Best model:  ARIMA(5,0,0)(0,0,0)[0] intercept
Total fit time: 2.598 seconds
Performing stepwise search to mi